# 라이브러리 로딩

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import os
from PIL import Image
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import random
import time

from sklearn.model_selection import train_test_split

from randaugment import RandAugment 

# 시드고정


In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything()

In [4]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img_path = './train_2/'+img_path.split('/')[-1]
        image = Image.open(img_path).convert('RGB')
        
        if self.transforms is not None:
            image = self.transforms(image)
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [5]:
def get_train_val(df):
    df = df.sample(frac=1)
    train_len = int(len(df) * 0.8)
    train_df = df[:train_len]
    val_df = df[train_len:]
    
    train_x, train_y  = train_df['img_path'].to_list(), train_df.iloc[:,2:].values
    val_x, val_y = val_df['img_path'].to_list(), val_df.iloc[:,2:].values
 
    return train_x,train_y, val_x,val_y


def get_dataloader(df,mode,batch_size):
    """
    데이터프레임을 dataloader형태로 반환하는 함수
    """
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    # augmentations 적용 
    train_augmentations = transforms.Compose([   
                    transforms.Resize((224, 224)),
                    #transforms.RandomResizedCrop(224),
#                     transforms.RandomCrop(224),
#                     transforms.RandomRotation(30),
#                     transforms.RandomGrayscale(p=0.4),
#                     transforms.Grayscale(num_output_channels=3),
#                     transforms.RandomAffine(45, shear=0.2),
#                     transforms.ColorJitter(),
                    RandAugment(), 
                    transforms.RandomHorizontalFlip(),
                    #transforms.Lambda(utils.randomColor),
                    #transforms.Lambda(utils.randomBlur),
                    #transforms.Lambda(utils.randomGaussian),
                    transforms.ToTensor(),
                    normalize,
            ])
    valid_augmentations = transforms.Compose([
                    transforms.Resize((224, 224)),
                    #transforms.CenterCrop(299),
                    #transforms.Grayscale(num_output_channels=3),
                    transforms.ToTensor(),
                    normalize
        ])
    
    if mode == 'TRAIN':

        train_x,train_y, val_x,val_y = get_train_val(df)

        # Data Loader
        train_dataset = CustomDataset(train_x,train_y, transforms = train_augmentations)
        valid_dataset = CustomDataset(val_x, val_y, transforms = valid_augmentations)
        
        train_data_loader = DataLoader(
            train_dataset,
            batch_size = batch_size,
            shuffle = True,
            num_workers = 16,
            pin_memory = True
        )
        valid_data_loader = DataLoader(
            valid_dataset,
            batch_size = batch_size,
            shuffle = False,
            num_workers = 16,
            pin_memory = True
        )
        return train_data_loader, valid_data_loader
    
    else :
        test_dataset = CustomDataset( img_path_list = df['img_path'], transforms = valid_augmentations)
        test_data_loader = DataLoader(
            test_dataset,
            batch_size = batch_size,
            shuffle = False,
            num_workers = 16,
            drop_last = False
        )
        return test_data_loader
        

    

In [6]:
def train_model(model, dataloaders_dict, optimizer, num_epochs, device, model_type):
    """
    train 데이터로 모델을 학습하고 valid 데이터로 모델을 검증하는 코드

    파라미터
    ---
    model : 
        학습할 모델
    dataloaders_dict : dict
        train_dataloader과 validation_datalodaer가 들어 있는 dictonary
    optimizer : 
        최적화 함수
    num_epochs : int
        학습 횟수
    device : cuda or cpu
        모델을 학습할 때 사용할 장비
    model_type : m or l or dense 
        모델 back born

    returns 
    best_model :
        검증데이터 셋 기준으로 가장 성능이 좋은 모델
    ---
    """
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=5,
                                                   gamma=0.9)
    model.to(device)
    torch.cuda.empty_cache()
    critrion = torch.nn.CrossEntropyLoss()
    # 학습이 어느정도 진행되면 gpu 가속화
    #torch.backends.cudnn.benchmark = False

    # loss가 제일 낮은 모델을 찾기위한 변수
    best_val_loss = int(1e9)
    for epoch in range(num_epochs):
        # epoch 별 학습 및 검증

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로
            else:
                model.eval()   # 모델을 추론 모드로

            epoch_loss = [] # epoch loss
            epoch_acc = []  # epoch 정확도
            for i,  (imgs, labels) in enumerate(dataloaders_dict[phase]):
                images, labels = imgs, labels
                # tensor를 gpu에 올리기
                images = images.to(device)
                labels = labels.to(device)
                

                # 옵티마이저 초기화 초기화
                optimizer.zero_grad()

                # 순전파 계산
                with torch.set_grad_enabled(phase == 'train'):
                    probs = model(images)
                    loss = critrion(probs, labels)
                    
                    # 학습시 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                     
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
        
                    epoch_loss.append(loss.item())
                    epoch_acc.append(batch_acc)
                    
            # epoch별 loss 및 정확도
            epoch_loss = np.mean(epoch_loss)
            epoch_acc = np.mean(epoch_acc)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

            # 검증 오차가 가장 적은 최적의 모델을 저장
            if phase=='val' and epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                best_model = model
                torch.save(best_model.state_dict(), './checkpoint/'+'{}_{}_{}.pth'.format(model_type,epoch,best_val_loss))
                
            lr_scheduler.step()

    return best_model

In [7]:
class BaseModel(nn.Module):
    def __init__(self, model_type, num_classes=10):
        super(BaseModel, self).__init__()
        if model_type == 'm' :
            self.backbone = models.efficientnet_v2_m(pretrained=True)
        elif model_type == 'l' :
            self.backbone = models.efficientnet_v2_l(pretrained=True)  
        elif model_type =='dense':
            self.backbone = models.densenet201(pretrained=True)  
    
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

# 모델 학습

In [ ]:
dataframe = pd.read_csv('open/train.csv')
best_models = []
# 교차 검증을 진행할 K
K = 10
for model_type in ['m','l','dense']:
    for i in range(K):

        train_loader, val_loader = get_dataloader(
            dataframe, mode='TRAIN', batch_size=16)


        #  dict 형식으로 data loader 정의
        dataloaders_dict = {"train": train_loader, "val": val_loader}
        num_epochs = 200
        max_grad_norm = 1
        learning_rate = 0.0005

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print('사용하는 device :', device)
        print('--------------------', i+1, '/', K,
              '- fold start--------------------')

        model = BaseModel(model_type)

        optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

        best_model = train_model(model, dataloaders_dict,
                                 optimizer, num_epochs, device, model_type)
        best_models.append(best_model)